## Imports

In [2]:
import ibis
from ibis import _
ibis.options.interactive = True
import ibis.selectors as s
import os

## Functions to run the TPC-H queries

In [3]:
def tpc_h01(con):
    """    
    This query reports the amount of business that was billed, shipped, and returned.
    
    The Pricing Summary Report Query provides a summary pricing report for all lineitems
    shipped as of a given date. The date is within 60 - 120 days of the greatest ship date
    contained in the database. The query lists totals for extended price, discounted extended
    price, discounted extended price plus tax, average quantity, average extended
    price, and average discount. These aggregates are grouped by RETURNFLAG and LINESTATUS,
    and listed in ascending order of RETURNFLAG and LINESTATUS. A count of the number of
    lineitems in each group is included.
    """
    lineitem = con.table("lineitem")
    q01 = (
        lineitem
        .filter(_.l_shipdate <= (ibis.date('1998-12-01') - ibis.interval(days=90)))
        .aggregate(
            by=[_.l_returnflag, _.l_linestatus],
            sum_qty=_.l_quantity.sum(),
            sum_base_price=_.l_extendedprice.sum(),
            sum_disc_price=(_.l_extendedprice * (1 - _.l_discount)).sum(),
            sum_charge=(_.l_extendedprice * (1 - _.l_discount) * (1 + _.l_tax)).sum(),
            avg_qty=_.l_quantity.mean(),
            avg_price=_.l_extendedprice.mean(),
            avg_disc=_.l_discount.mean(),
            count_order=_.count()
        )
        .order_by([_.l_returnflag, _.l_linestatus])
    )
    return q01


def tpc_h10(con):
    """
    The query identifies customers who might be having problems with the parts that are
    shipped to them.
    
    The Returned Item Reporting Query finds the top 20 customers, in terms of their effect
    on lost revenue for a given quarter, who have returned parts. The query considers only
    parts that were ordered in the specified quarter. The query lists the customer's name,
    address, nation, phone number, account balance, comment information and revenue lost.
    The customers are listed in descending order of lost revenue. Revenue lost is defined
    as sum(l_extendedprice*(1-l_discount)) for all qualifying lineitems.
    """
    lineitem = con.table('lineitem')
    orders = con.table('orders')
    customer = con.table("customer")
    nation = con.table("nation")
    
    q = (
        customer
            .join(orders, orders.o_custkey == customer.c_custkey)
            .join(lineitem, lineitem.l_orderkey == orders.o_orderkey)
            .join(nation, customer.c_nationkey == nation.n_nationkey)
    )

    q = q.filter(
        [
            (q.o_orderdate >= ibis.date("1993-10-01")) & (q.o_orderdate < (ibis.date("1993-10-01") + ibis.interval(months=3))),
            q.l_returnflag == "R",
        ]
    )

    gq = q.group_by(
        [
            q.c_custkey,
            q.c_name,
            q.c_acctbal,
            q.c_phone,
            q.n_name,
            q.c_address,
            q.c_comment,
        ]
    )
    q = gq.aggregate(revenue=(q.l_extendedprice * (1 - q.l_discount)).sum())

    q = q.order_by(ibis.desc(q.revenue))
    return q.limit(20)


## Postgres

In [4]:
pg_user = os.environ.get("PG_USER")
pg_pwd = os.environ.get("PG_PWD")

con_pg = ibis.postgres.connect(database='tpc-h-10gb', host='localhost', user=pg_user, password=pg_pwd)

### Example TPC-H query results

In [5]:
tpc_h01(con_pg)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ l_returnflag ┃ l_linestatus ┃ sum_qty        ┃ sum_base_price   ┃ sum_disc_price     ┃ sum_charge           ┃ avg_qty        ┃ avg_price      ┃ avg_disc       ┃ count_order ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string       │ string       │ decimal(38, 2) │ decimal(38, 2)   │ decimal(38, 4)     │ decimal(38, 6)       │ decimal(15, 2) │ decimal(15, 2) │ decimal(15, 2) │ int64       │
├──────────────┼──────────────┼────────────────┼──────────────────┼────────────────────┼──────────────────────┼────────────────┼────────────────┼────────────────┼─────────────┤
│ A            │ F            │   377518399.00 │  566065727797.25 │  537759104278.0656 │  559276670892.116821 │          25.50 │       38237.15 │           0.05 │    14804077 │
│ N            │ F            │     9851614.00 │   14767438399.17 │   14028805792.2114 │   14590490998.366737 │          25.52 │       38257.81 │           0.05 │      385998 │
│ N            │ O            │   743124873.00 │ 1114302286901.88 │ 1058580922144.9637 │ 1100937000170.591797 │          25.50 │       38233.90 │           0.05 │    29144351 │
│ R            │ F            │   377732830.00 │  566431054976.00 │  538110922664.7677 │  559634780885.086304 │          25.51 │       38251.22 │           0.05 │    14808183 │
└──────────────┴──────────────┴────────────────┴──────────────────┴────────────────────┴──────────────────────┴────────────────┴────────────────┴────────────────┴─────────────┘

In [18]:
tpc_h10(con_pg)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ c_custkey ┃ c_name             ┃ c_acctbal      ┃ c_phone         ┃ n_name                    ┃ c_address                           ┃ c_comment                                                                        ┃ revenue        ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ !int32    │ string             │ decimal(15, 2) │ string          │ string                    │ string                              │ string                                                                           │ decimal(38, 4) │
├───────────┼────────────────────┼────────────────┼─────────────────┼───────────────────────────┼─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┼────────────────┤
│   1237537 │ Customer#001237537 │        7840.17 │ 32-367-120-4327 │ RUSSIA                    │ FNG6WgB1mopyyY,ajQTU qUPW5o         │ nag carefully about the regular packages. carefully reg                          │    884989.6657 │
│   1116802 │ Customer#001116802 │         772.17 │ 23-199-655-3770 │ JORDAN                    │ JpGQT3IHoogqB3ZMrSrzhBrVFObr        │ ests. quickly ironic deposits along the special requests a                       │    830214.1117 │
│    508954 │ Customer#000508954 │        3072.07 │ 15-880-510-9487 │ ETHIOPIA                  │ zgafElAXecB3I7Ee1a9B                │ ickly regular deposits. carefully regular instructions engage deposits. foxes a… │    826167.5138 │
│   1487503 │ Customer#001487503 │        5398.11 │ 20-271-234-4401 │ IRAN                      │ orychjo, O6c                        │ ccounts sleep carefully. furiously pending packages boost slyly after the quick… │    825996.5903 │
│   1461247 │ Customer#001461247 │        3205.93 │ 30-137-949-5375 │ SAUDI ARABIA              │ jQZKjwNqttmC0BohJoGA4U              │ atterns sleep blithely. fluffily pending accounts sleep furiously from the iron… │    775992.0820 │
│   1316338 │ Customer#001316338 │        8682.95 │ 18-793-610-3832 │ INDIA                     │ bfz3FH  qaULsV7oiCn66z6ezkZBpcl     │ equests. regular, even excuses doze slyly against the blithely even accounts. s… │    765063.7003 │
│    817849 │ Customer#000817849 │        -380.30 │ 25-111-847-3201 │ MOROCCO                   │ 7gz7HbNoVNSbv23trx                  │ fully special accounts wake slyly among the sheaves. permanently brave requests… │    737664.7232 │
│   1335394 │ Customer#001335394 │        3978.53 │ 31-760-271-2302 │ VIETNAM                   │ mdNmISybrVNaIPSp                    │  slyly even instructions. carefully silent ideas                                 │    731259.8675 │
│    102358 │ Customer#000102358 │        3564.90 │ 12-941-212-8350 │ BRAZIL                    │ YpDpv0sP14F5xMUQbkDV,XGcMJypFV6jH   │ y slyly regular multipliers. furiously regular deposits nag final excuses. slyl… │    715001.8825 │
│   1432624 │ Customer#001432624 │         205.08 │ 16-550-232-2774 │ FRANCE                    │ NTCV7qvJ0PgcULFci,VdAB,0ZbSEZRdnnz, │ ctions use regular requests. quickly bold somas cajole at the c                  │    713512.6431 │
│         … │ …                  │              … │ …               │ …                         │ …                                   │ …                                                                                │              … │
└───────────┴────────────────────┴────────────────┴─────────────────┴───────────────────────────┴─────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────┴──────

### TPC-H queries benchmarking on Postgres

In [6]:
%%timeit -r10
tpc_h01(con_pg).execute()

27 s ± 9.39 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [25]:
%%timeit -r10
tpc_h10(con_pg).execute()

5.69 s ± 162 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## DuckDB

In [20]:
con_ibis = ibis.duckdb.connect()
con_ibis.register("/home/francois/datasets/tpc-h-10GB/parquet/lineitem.parquet", table_name="lineitem")
con_ibis.register("/home/francois/datasets/tpc-h-10GB/parquet/orders.parquet", table_name="orders")
con_ibis.register("/home/francois/datasets/tpc-h-10GB/parquet/customer.parquet", table_name="customer")
con_ibis.register("/home/francois/datasets/tpc-h-10GB/parquet/nation.parquet", table_name="nation")
con_ibis.list_tables()

['nation', 'customer', 'orders', 'lineitem']

### TPC-H queries benchmarking on DuckDB

In [23]:
%%timeit -r 10
tpc_h01(con_ibis).execute()

3.48 s ± 69.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [26]:
%%timeit -r 10
tpc_h10(con_ibis).execute()

3.99 s ± 250 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "25g").appName("tpc-h-10GB").getOrCreate()

In [5]:
con_spark = ibis.pyspark.connect(spark)

In [33]:
con_spark.register("/home/francois/datasets/tpc-h-10GB/parquet/lineitem.parquet", table_name="lineitem")
con_spark.register("/home/francois/datasets/tpc-h-10GB/parquet/orders.parquet", table_name="orders")
con_spark.register("/home/francois/datasets/tpc-h-10GB/parquet/customer.parquet", table_name="customer")
con_spark.register("/home/francois/datasets/tpc-h-10GB/parquet/nation.parquet", table_name="nation")

┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ n_nationkey ┃ n_name    ┃ n_regionkey ┃ n_comment                                                                        ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ string    │ int64       │ string                                                                           │
├─────────────┼───────────┼─────────────┼──────────────────────────────────────────────────────────────────────────────────┤
│           0 │ ALGERIA   │           0 │  haggle. carefully final deposits detect slyly agai                              │
│           1 │ ARGENTINA │           1 │ al foxes promise slyly according to the regular accounts. bold requests alon     │
│           2 │ BRAZIL    │           1 │ y alongside of the pending deposits. carefully special packages are about the i… │
│           3 │ CANADA    │           1 │ eas hang ironic, silent packages. slyly regular packages are furiously over the… │
│           4 │ EGYPT     │           4 │ y above the carefully unusual theodolites. final dugouts are quickly across the… │
│           5 │ ETHIOPIA  │           0 │ ven packages wake quickly. regu                                                  │
│           6 │ FRANCE    │           3 │ refully final requests. regular, ironi                                           │
│           7 │ GERMANY   │           3 │ l platelets. regular accounts x-ray: unusual, regular acco                       │
│           8 │ INDIA     │           2 │ ss excuses cajole slyly across the packages. deposits print aroun                │
│           9 │ INDONESIA │           2 │  slyly express asymptotes. regular deposits haggle slyly. carefully ironic hock… │
│           … │ …         │           … │ …                                                                                │
└─────────────┴───────────┴─────────────┴──────────────────────────────────────────────────────────────────────────────────┘

### TPC-H queries benchmarking on PySpark

In [21]:
%%timeit -r 10
tpc_h01(con_spark).execute()

7.46 s ± 125 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [35]:
%%timeit -r 10
tpc_h10(con_spark).execute()

7.87 s ± 688 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
